In [1]:
import os
import asyncio
import yaml
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_core.models import ChatCompletionClient, ModelInfo
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination, TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, UnstructuredWordDocumentLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [41]:
AZURE = "C:/Master IA/TFM_BA_GPT/src/model_config_azure.yaml"
OLLAMA = "C:/Master IA/TFM_BA_GPT/src/model_config_ollama.yaml"
LMSTUDIO = "C:/Master IA/TFM_BA_GPT/src/model_config_lmstudio.yaml"

# Load model configuration and create the model client.
with open(AZURE, "r") as f:
    model_config = yaml.safe_load(f)
    
model_client = ChatCompletionClient.load_component(model_config)

In [16]:
### --- CONFIG PATH--- ###
CHROMA_DIR = "../chromadb_storage"
DOCS_DIR = "../data"
COLLECTION_NAME = "rag_collection"

### --- EMBEDDING FUNCTION (Local SentenceTransformer) --- ###
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

In [17]:
# Step 2: Set up ChromaDB client with embedding function
chroma_client = chromadb.Client(Settings(
    persist_directory=CHROMA_DIR,
    anonymized_telemetry=False
))

# Step 3: Create or load the collection WITH embedding
collections = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_function
)

In [18]:
### --- RAG-ENABLED AUTO-GEN SETUP WITH CHROMADB --- ###

vector_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name=COLLECTION_NAME,
        persistence_path=CHROMA_DIR,
        k=3,
        score_threshold=0.4,
    )
)

In [19]:
#Funtion to load and index documents
# This function loads documents from a specified directory, splits them into chunks,
# and indexes them in the provided vector memory.
# It supports PDF, TXT, and DOCX formats.
# The function uses a text splitter to divide the documents into smaller chunks
# based on the specified chunk size and overlap.
# It also handles exceptions for unsupported file formats and errors during processing.

async def load_and_index_documents(docs_dir, vector_memory, chunk_size=500, chunk_overlap=50):
    """
    Loads PDF, TXT, and DOCX documents from the specified directory,
    splits them into chunks, and indexes them in the provided AutoGen vector memory.

    Args:
        docs_dir (str): Path to the directory containing the documents.
        vector_memory (ChromaDBVectorMemory): The vector memory object to index the documents into.
        chunk_size (int): Number of characters in each chunk.
        chunk_overlap (int): Number of overlapping characters between chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    for file_name in os.listdir(docs_dir):
        file_path = os.path.join(docs_dir, file_name)
        ext = os.path.splitext(file_name)[-1].lower()

        try:
            if ext == ".pdf":
                loader = PyMuPDFLoader(file_path)
            elif ext == ".txt":
                loader = TextLoader(file_path)
            elif ext == ".docx":
                loader = UnstructuredWordDocumentLoader(file_path)
            else:
                print(f"Unsupported file format: {file_name}")
                continue

            docs = loader.load()
            split_docs = text_splitter.split_documents(docs)

            for i, chunk in enumerate(split_docs):
                await vector_memory.add(  # Add `await` here
                    MemoryContent(
                        content=chunk.page_content,
                        mime_type=MemoryMimeType.TEXT,
                        metadata={"source": file_name, "chunk_index": i}
                    )
                )

            print(f"Indexed {len(split_docs)} chunks from {file_name}.")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")
    return chunk_size, chunk_overlap

In [20]:
async def load_and_index_web_page(
    urls,
    vector_memory,
    chunk_size=500,
    chunk_overlap=50
):
    """
    Loads one or multiple web pages from the specified URL(s), splits their content into chunks,
    and indexes the chunks in the provided AutoGen vector memory.

    Args:
        urls (str or list): The URL or list of URLs of the web pages to load.
        vector_memory: The vector memory object to index the web page content.
        chunk_size (int): Number of characters in each chunk.
        chunk_overlap (int): Number of overlapping characters between chunks.
    Returns:
        tuple: The chunk_size and chunk_overlap used.
    """
    # Ensure that urls is a list even if a single URL is provided.
    if isinstance(urls, str):
        urls = [urls]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    async def process_url(url):
        try:
            # Load the web page content.
            loader = WebBaseLoader(url)
            docs = loader.load()
            # Split the loaded documents into chunks.
            split_docs = text_splitter.split_documents(docs)

            for i, chunk in enumerate(split_docs):
                await vector_memory.add(
                    MemoryContent(
                        content=chunk.page_content,
                        mime_type=MemoryMimeType.TEXT,
                        metadata={"source": url, "chunk_index": i}
                    )
                )

            print(f"Indexed {len(split_docs)} chunks from {url}.")

        except Exception as e:
            print(f"Error processing {url}: {e}")

    # Process all URLs concurrently.
    tasks = [process_url(url) for url in urls]
    await asyncio.gather(*tasks)

    return chunk_size, chunk_overlap


In [21]:
# Load and index documents and web pages if the directory does not exist.

sites = [
    "https://www.morebusiness.com/business-analyst-interview-questions/",
    "https://ellogy.ai/comprehensive-guide-asking-the-right-questions-in-it-requirements-gathering/",
    "https://medium.com/@alishadhillon__/typical-requirements-gathering-questions-in-a-data-viz-project-fa7c04f0b2ad",
    "https://www.requiment.com/the-most-asked-questions-about-requirements-gathering/",
    "https://practicalanalyst.com/requirements-elicitation-most-valuable-questions/",
    "https://www.modernanalyst.com/Resources/Articles/tabid/115/ID/179/8-Questions-Every-Business-Analyst-Should-Ask.aspx",
    "https://www.thenarratologist.com/best-business-analyst-questions-to-gather-requirements/",
]

if not os.path.exists(CHROMA_DIR):
    await vector_memory.clear()
    print("Indexing documents... please wait.")
    await load_and_index_documents(DOCS_DIR, vector_memory)
    await load_and_index_web_page(sites, vector_memory)

In [9]:
collection = vector_memory._collection
collection.peek(5)

AttributeError: 'NoneType' object has no attribute 'peek'

In [10]:
#Set up the Rag-enabled AutoGen agent and the user proxy
# The user proxy is a special agent that represents the human user in the conversation.
# It can be used to send messages to the assistant agent and receive responses.
# The assistant agent is the main agent that interacts with the user and performs tasks.

user_proxy = UserProxyAgent(
    name="user_proxy",
    description="A human user",
    input_func=input
)

assistant_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    memory=[vector_memory],
    system_message="You are a helpful assistant that use RAG to answer questions"  # if you have a custom system prompt
)


In [11]:
#Set up the termination word for the conversation
# The termination condition is a special condition that indicates when the conversation should end.
# In this case, the conversation will end when the user types "exit".
termination = TextMentionTermination("exit")

# Create a round-robin group chat team with both agents
team = RoundRobinGroupChat(
    [assistant_agent, user_proxy], 
    termination_condition=termination
)

In [ ]:
# stream = assistant_agent.run_stream(task="what are the  books that all serious practitioners will have on their bookshelves?")
# await Console(stream)

# await vector_memory.close()
# await model_client.close()

---------- user ----------
what are the  books that all serious practitioners will have on their bookshelves?
---------- assistant ----------
[MemoryContent(content='about programming, there will be lots of code. If the book is about managing, there \nwill be lots of case studies from real projects. \nThese are the books that all serious practitioners will have on their bookshelves. \nThese are the books that will be remembered for making a difference and for guiding \nprofessionals to become true craftsman. \nManaging Agile Projects\nSanjiv Augustine\nAgile Estimating and Planning\nMike Cohn\nWorking Effectively with Legacy Code\nMichael C. Feathers', mime_type='MemoryMimeType.TEXT', metadata={'chunk_index': 3, 'mime_type': 'MemoryMimeType.TEXT', 'source': 'clean-code.pdf', 'score': 0.5309425592422485, 'id': '526b59c4-e355-42f4-b3f7-fbcb55ccec6f'}), MemoryContent(content='making readable, 311\nnecessity of, 2\nreading from top to bottom, 37\nsimplicity of, 18, 19\ntechnique for shroud

In [12]:
# Function to run the chat

initial_question = "tell me about NFRs - Assumptions"
stream = team.run_stream(task=initial_question)
await Console(stream)  # stream the conversation to console

---------- user ----------
tell me about NFRs - Assumptions
---------- assistant ----------
[MemoryContent(content='NFRs - Assumptions \n• \nAssumption No - The number for the assumption being made sequential number in the \nformat ASSM-01, ASSM-02 etc. \n• \nDescription - The documented assumption. \n• \nStatus - The Status of the assumption could be Documented or Validated. \no This means that when the assumption is first posed it should be documented as \nbeing on further investigation and analysis it will become evident if the \nassumptions are Validated or not.', mime_type='MemoryMimeType.TEXT', metadata={'source': 'NFRs Assumptions.doc.pdf', 'chunk_index': 0, 'mime_type': 'MemoryMimeType.TEXT', 'score': 0.7519146800041199, 'id': 'cdfe665b-51ac-4cfd-815f-2b1348b4b99c'}), MemoryContent(content='identify how far from success a project might be. \nGreat non-functional requirements can be instrumental to a project’s success in many different \nways aside from being a success measure. 

c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:920: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


Non-functional requirements (NFRs) define criteria that can be used to judge the operation of a system, outside its functional behaviors, and often include considerations such as performance, security, reliability, usability, and scalability. Within NFRs, **assumptions** play a vital role in outlining and documenting specific expectations or conditions that are taken for granted during the analysis and design of a system.

### Key Components of NFRs - Assumptions:
1. **Assumption Number**:
   - Each assumption is assigned a unique identifier in the format **ASSM-01, ASSM-02**, etc., to ensure traceability and easy reference.

2. **Description**:
   - The assumption itself is clearly documented, explicitly stating the condition or premise being made.

3. **Status**:
   - The status of an assumption can be **Documented** or **Validated**.
     - **Documented**: The assumption is noted but not yet confirmed.
     - **Validated**: Analysis and investigation have supported the correctness a

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='tell me about NFRs - Assumptions', type='TextMessage'), MemoryQueryEvent(source='assistant', models_usage=None, metadata={}, content=[MemoryContent(content='NFRs - Assumptions \n• \nAssumption No - The number for the assumption being made sequential number in the \nformat ASSM-01, ASSM-02 etc. \n• \nDescription - The documented assumption. \n• \nStatus - The Status of the assumption could be Documented or Validated. \no This means that when the assumption is first posed it should be documented as \nbeing on further investigation and analysis it will become evident if the \nassumptions are Validated or not.', mime_type='MemoryMimeType.TEXT', metadata={'source': 'NFRs Assumptions.doc.pdf', 'chunk_index': 0, 'mime_type': 'MemoryMimeType.TEXT', 'score': 0.7519146800041199, 'id': 'cdfe665b-51ac-4cfd-815f-2b1348b4b99c'}), MemoryContent(content='identify how far from success a project might be. \nGreat no

In [24]:
await vector_memory.close()
await model_client.close()

SWARM with agents

In [42]:
user_proxy = UserProxyAgent(
        name="user",
        description="A human user",
        input_func=input,
    )

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    memory=[vector_memory],
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Interviewer: ask the user questions to gather information
        summarizer: summarize all the information provided and ask the user for feedback
        user: the human user who will provide information and feedback

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After assigning tasks, wait for all agents to finish their tasks.
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

interviewer_agent = AssistantAgent(
        name="interviewer",
        model_client=model_client,
        memory=[vector_memory],
        system_message=(
            "You are a Business Analyst."
            "You will ask the user questions one by one to gather information about"
            "the scope and requirements for a project."
            "You will use the RAG memory to get help and context to ask questions"
        ),
        model_client_stream=True,  # Enable model client streaming.
    )

summarizer_agent = AssistantAgent(
        name="summarizer",
        model_client=model_client,
        system_message=(
            "You are an expert on making summaries."
            "Summarize the the information provided by the interviewer then"
            "handoff the summary to the user and ask the user if he wants to add more information,"
            "if the user says yes, you will handoff the conversation to the interviewer"
            "and let him know the user have more information."
        ),
        model_client_stream=True,  # Enable model client streaming.
    )

In [43]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [44]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [45]:
team = SelectorGroupChat(
    [planning_agent, interviewer_agent, summarizer_agent, user_proxy],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [46]:
task = "Write the functional specifications for a new product"

In [47]:
await Console(team.run_stream(task=task))

c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_selector_group_chat.py:195: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)


---------- user ----------
Write the functional specifications for a new product
---------- PlanningAgent ----------
[MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDocument Status IN PROGRESS ONHOLD COMPLETED \nAuthor(s) \n[BA name] \nSponsor \n[Sponsor name] \nRelated Area \n[area related to the sponsor] \n \n1. Document Information \n \n1.1. Document Approvers & Reviewers \nName \nRole \nApprover / \nReviewer \nApproval / \nReview Date \nApproved / \nReview Version \nHead of UAT \nHead of UAT \nApprover \n  \n  \nHead of Business \nChange', mime_type='MemoryMimeType.TEXT', metadata={'source': 'Functional Requirements Specification Template.doc.pdf', 'chunk_index': 0, 'mime_type': 'MemoryMimeType.TEXT', 'score': 0.5851926207542419, 'id': '9327482c-df3d-47c5-ace0-37974cdbd6d5'}), MemoryContent(content='• \nFunctional Requirements Specification \n• \n1. Document Information  \n

c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:920: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- interviewer ----------
[MemoryContent(content='• \nFunctional Requirements Specification \n• \n1. Document Information  \no 1.1. Document Approvers & Reviewers \no 1.2. Document History \no 1.3. Reference Documents \no 1.4. Definitions, Acronyms and Abbreviations \n• \n2. Executive Summary  \no 2.1. Overview \no 2.2. Business Benefits \n• \n3. Scope  \no 3.1. In Scope \no 4.2. Out of Scope \no 3.3. Systems & Platforms LeanIX Meta-Data \n• \n4. Requirements  \no 4.1. Functional Requirements \no 4.2. Non-Functional Requirements \n• \n5. Functional Solution', mime_type='MemoryMimeType.TEXT', metadata={'mime_type': 'MemoryMimeType.TEXT', 'chunk_index': 3, 'source': 'Functional Requirements Specification Template.doc.pdf', 'score': 0.6396330296993256, 'id': 'f3ed4023-7dbf-4136-9390-90086da133d9'}), MemoryContent(content='Functional Requirements Specification - \nTemplate \n \nFunctional Requirements Specification \nBS-[number] [title] \nDOCUMENTATION \nDocument Status IN PROGRESS

Error processing publish message for SelectorGroupChatManager_5ea11a52-a0d6-4afd-b20b-f6f4aa0d091d/5ea11a52-a0d6-4afd-b20b-f6f4aa0d091d
Traceback (most recent call last):
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 533, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
    )
    ^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_core\_routed_agent.py", line 485, in on_message_impl
    return

RuntimeError: InternalServerError: Error code: 500 - {'error': {'message': 'The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently.', 'type': 'model_error', 'param': None, 'code': None}}
Traceback:
Traceback (most recent call last):

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_base_group_chat_manager.py", line 189, in handle_agent_response
    speaker_name = await speaker_name_future
                   ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_selector_group_chat.py", line 174, in select_speaker
    agent_name = await self._select_speaker(roles, participants, history, self._max_selector_attempts)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Master IA\TFM_BA_GPT\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_selector_group_chat.p